In [ ]:
#!/usr/bin/env python

"""
Insert data into HBase with a Python script.

To create the table, first use the hbase shell. We are going to create a
namespace called "sample_data". The table for this script is called "rfic",
as we will be inserting Request for Information Cases from the City of
Indianapolis.

Our table will have only one column family named "data", and we are accepting
all table defaults.

Original data source
https://data.indy.gov/dataset/Request-for-Information-Cases/ts4b-8qa9

% hbase shell
hbase> create_namespace "sample_data"
hbase> create "sample_data:rfic", "data"
"""

import csv
import happybase
import time

#batch_size = 1
host = "localhost"
namespace = "sample_data"
row_count = 0
start_time = time.time()
table_name = "faq"

import quesanswer as qa

#ques , answer = qa.main()

#print(len(ques))

#for i in range (0,len(ques)):
#    print( i,". " ,ques[i], "\n" ,answer[i],"\n\n")

def connect_to_hbase():
    """ Connect to HBase server.

    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    conn = happybase.Connection(host = host,
        table_prefix_separator = ":")
    conn.open()
    batch = conn.table(table_name)
    #batch = table.batch(batch_size = batch_size)
    return conn , batch


def insert_row(table):
    """ Insert a row into HBase.

    Write the row to the batch. When the batch size is reached, rows will be
    sent to the database.

    Rows have the following schema:
        [ id, keyword, subcategory, type, township, city, zip, council_district,
          opened, closed, status, origin, location ]
    """
    for i in range (0,len(ques)):
        table.put(str(i+1),{'question:ques':ques[i],'question:ansindex':str(i+1),'answer:ans':answer[i]})
        print("Row - ", i ," Updated")

def retrieve_ques(table):
    def conv(Rvalue,ques,ansindex):
        key = []
        value =[]
        index = []
        for x in Rvalue:
            ky = x[0].decode()
            val = x[1][ques].decode()
            indx = x[1][ansindex].decode()
            key.append(ky)
            value.append(val)
            index.append(indx)
            
        #print(key)
        #print(value)
        #print(value)
        #print(index)
        return list(zip(value ,index))
    
    ques = b'question:ques'
    ansindex = b'question:ansindex'
    value = table.scan(columns=[ques,ansindex])
    return conv(value,ques,ansindex)

def retrieve_ans(table,index):
    
    rows = table.row(index)
    test='answer:ans'.encode()
    row = rows[test]
    print(row.decode())
    
    ''' 
    def conv(Rvalue,quali):
        key = []
        value =[]
        for x in Rvalue:
            ky = x[0].decode()
            val = x[1][quali].decode()
            key.append(ky)
            value.append(val)
            
        #print(key)
        #print(value)
        return value
    
    quali = b'question:ques'
    value = table.scan(columns=[quali])
    return conv(value,quali)
    '''
def getdata(typ,index='1'):
    try:                   
        #row = ['3','Good aerodynamics']
        # After everything has been defined, run the script.
        conn,batch = connect_to_hbase()
        #print(type(row))
        #insert_row(batch)
        if 'ans' in typ:
            return retrieve_ans(batch,index)
        else:
            return retrieve_ques(batch)
    finally:
        print('closed')
        conn.close()

    duration = time.time() - start_time
    print ("Done. row count: %i, duration: %.3f s" % (row_count, duration))


#question, ansindex = getdata('ques')
#getdata('ans','281')
    